In [1]:
!pip3 install slack_sdk==3.21.0

In [2]:
!pip3 install arxiv==1.4.4

In [3]:
!pip3 install openai==0.27.4

# SlackとopenaiのGPTで論文の要約をする

## Reference
- [最新の論文をChatGPTで要約して毎朝Slackに共有してくれるbotを作る！](https://zenn.dev/ozushi/articles/ebe3f47bf50a86)
- [Slack API を使用してメッセージを投稿する](https://zenn.dev/kou_pg_0131/articles/slack-api-post-message)
- [【Slack】インストールするボットユーザーがありませんと出たときの対処方法](https://the-simple.jp/slack-nobotuser#Step1Bot)

In [4]:
import os
import random
import time

from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import arxiv
import openai

In [5]:
#OpenAIのapiキー
openai.api_key = 'OPEN_API_KEY'
# Slack APIトークン

SLACK_API_TOKEN = 'SLACK_API_TOKEN'    # ユーザーとして API を実行するためのトークン 
# Slackに投稿するチャンネル名を指定する
SLACK_CHANNEL = "SLACK_CHANNEL"

In [6]:
def get_summary(result):
    system = """与えられた論文の全体を128文字以内にまとめた後、タイトルと原文とその日本語訳、論文リンク、著者/所属機関、投稿日付 (yyyy/MM/dd)、概要、新規性や差分、独特の手法、実験結果を以下のフォーマットで日本語で出力してください。```
    # タイトルの日本語訳
    ## 一言でいうと
    ---
    ### 概要
    ### 新規性・差分
    ### 手法
    ### 結果
    ### コメント
    ```"""

    text = f"title: {result.title}\nbody: {result.summary}"
    response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {'role': 'system', 'content': system},
                    {'role': 'user', 'content': text}
                ],
                temperature=0.25,
            )
    summary = response['choices'][0]['message']['content']
    title_en = result.title
    title, *body = summary.split('\n')
    body = '\n'.join(body)
    date_str = result.published.strftime("%Y-%m-%d %H:%M:%S")
    message = f"発行日: {date_str}\n{result.entry_id}\n{title_en}\n{title}\n{body}\n"
    
    return message


In [7]:
# Slack APIクライアントを初期化する
client = WebClient(token=SLACK_API_TOKEN)
#queryを用意
query_list = ['ti:%22 Anomaly Detection %22', 'ti:%22 AIOps %22']
message_list = ['Anomaly Detection', 'AIOps']

In [ ]:
for j in range(len(query_list)):
    query = query_list[j]
    # arxiv APIで最新の論文情報を取得する
    search = arxiv.Search(
        query=query,  # 検索クエリ（
        max_results=100,  # 取得する論文数
        sort_by=arxiv.SortCriterion.SubmittedDate,  # 論文を投稿された日付でソートする
        sort_order=arxiv.SortOrder.Descending,  # 新しい論文から順に取得する
    )
    
    #searchの結果をリストに格納
    result_list = []
    for result in search.results():
        result_list.append(result)

    #ランダムにnum_papersの数だけ選ぶ
    num_papers = 5
    results = random.sample(result_list, k=num_papers)
    
    # 論文情報をSlackに投稿する
    for i, result in enumerate(results):
        try:
            today = time.strftime('%Y-%m-%d', time.localtime())
            # Slackに投稿するメッセージを組み立てる
            message =  "本日 " + str(today) + f"{message_list[j]} の" + "論文 " + str(i+1) + "本目です\n" + get_summary(result)
            # Slackにメッセージを投稿する
            response = client.chat_postMessage(
                channel=SLACK_CHANNEL,
                text=message
            )
            print(f"Message posted: {response['ts']}")
        except SlackApiError as e:
            print(f"Error posting message: {e}")

Message posted: 1681223115.977069
